In [155]:
import nltk 
from nltk import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import numpy as np
import re
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,SimpleRNN 
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as utils
import re
from keras.callbacks import ModelCheckpoint
import math
import codecs
import random
from nltk.tokenize import sent_tokenize
import operator
import string
from random import randrange as rd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
file=open("speech_data.txt","r",encoding="UTF-8")
data=file.read()
file.close()
data = re.sub('SPEECH \d+', '\n', data)
data = re.sub('’', "'",data)
data = data.replace('...',' ')
data = data.replace('..',' ')
data = re.sub("[^A-Za-z.']",' ',data)
data = data.replace('\n',' ')
data = data.replace('  ',' ')
data = data.replace('  ',' ')

In [0]:
tokens = sent_tokenize(data) 

In [0]:
def pad(tokens):
  temp=[]
  comp=[]
  for i in tokens:
    var=nltk.word_tokenize(i)
    var.append("</s>")
    var.insert(0,"<s>")
    temp.append(var)
    for j in var:
      comp.append(j)
  return(temp,comp)
    
  

In [0]:
def padding(lis):
  sent_list=[]
  token_list=[]
  for i in lis:
    i+="</s> " 
    i=i[::-1]
    i+=" >s<"
    i=i[::-1]
    sent_list.append(i)
    token_list.append(i.split(" "))
  return(sent_list,token_list)

In [0]:
sent_list, comp = pad(tokens)


In [0]:

train_1, test_1 = train_test_split(sent_list, test_size=0.2)

In [0]:
vocab_dict = {}
vocab_len = 0
for i in (train_1):
  for j in i:
    if(j in vocab_dict):
      vocab_dict[j] += 1
    else:
      vocab_dict[j] = 1
      vocab_len += 1

# Get n gram

In [0]:
def ngram(lis,n):
  poss={}
  for i in lis:
    for j in list(nltk.ngrams(i,n)):
      if(j in poss):
        poss[j]+=1
      else:
        poss[j]=1
  return(poss)

# Unigram

In [113]:
n=1
uni=ngram(train_2,n)
print("Number of Unigrams:", len(list(uni.keys())))

Number of Unigrams: 8877


# BiGram 

In [114]:
n=2
bi=ngram(train_2,n)
print("Number of Bigrams:", len(list(bi.keys())))

Number of Bigrams: 51157


# Trigram

In [115]:
n=3
tri=ngram(train_2,n)
print("Number of Trigrams:", len(list(tri.keys())))

Number of Trigrams: 90345


# Quadgrams

In [116]:
n=4
quad=ngram(train_2,n)
print("Number of quadgrams:", len(list(quad.keys())))

Number of quadgrams: 103023


# MLE

In [0]:
def unig(word):
  if(tuple(word) in uni):
    return((uni[tuple(word)]+1)/(len(list(uni.keys()))+vocab_len))
  else:
    return 1/vocab_len
def big(word):
  if(tuple(word) in bi):
    return ((bi[tuple(word)]+1))/(len(list(bi.keys()))+vocab_len)
    
  else:
      return 1/vocab_len
    
def trig(word):
  if(tuple(word) in tri):
    return (tri[tuple(word)]+1)/(len(list(tri.keys()))+vocab_len)
  else:
      return 1/vocab_len
            
def quadg(word):
  if(tuple(word) in quad):
    return (quad[tuple(word)]+1)/(len(list(quad.keys()))+vocab_len)
  else:
      return 1/vocab_len

In [0]:
updated_test=[]
for i in test_2:
  for j in i:
    updated_test.append(j)

In [117]:
import random 
# unigram_=random.sample(updated_test,1)
unigram_test=random.sample(updated_test,1)


# Bigram_=random.sample(updated_test,1)
Bigram_test=random.sample(updated_test,2)

# trigram_=random.sample(updated_test,1)
trigram_test=random.sample(updated_test,3)

# quadgram_=random.sample(updated_test,1)
quadgram_test=random.sample(updated_test,4)

print(unig(unigram_test))
print(big(Bigram_test))
print(trig(trigram_test))
print(quadg(quadgram_test))

0.00014814814814814815
0.00014814814814814815
0.00014814814814814815
0.00014814814814814815


In [0]:
test_string=""
for i in test_1:
  
  test_string+=i
  test_string+=" "

# perplexity

In [0]:
def Perprex(data,n):
  if(n==1):
    typp=unig
  elif(n==2):
    typp=big
  elif(n==3):
    typp=trig
  elif(n==4):
    typp=quadg
  prob=0
  tot_len=0
  for i in data:
    temp=0
    for j in range(len(i)-n+1):
      w = i[j:j+n]
      
      temp += math.log(typp(w),2)

    temp=math.pow(2,temp)

    prob -= math.log(temp,2)
    sentence = i.split(" ")
    tot_len += (len(sentence)-n-1)
  
  return math.pow(2, prob/tot_len)

In [22]:
print("unigram perplexity: ",Perprex(test_string, 1))
print("bigram perplexity: ",Perprex(test_string, 2))
print("trigram perplexity: ",Perprex(test_string, 3))
print("quadgram perplexity: ",Perprex(test_string, 4))

570.1665432242371
51.86543504814822
9.093378064057491
2.081737140439814


In [0]:
vocab_dict

# Generator

In [0]:
#Classical generator to generate new words using a n_gram model
def MLE(n_gram, initial_sequence):
  sentence = ["<s>"]
  if(n_gram==1):
    for i in range(20):
      max_prob = 0
      max_prob_list = list()
      for j in vocab_dict.keys():
        k = (unig([j]))
        if(k>max_prob):
          max_prob = k
          max_prob_list = [j]
        elif(k==max_prob):
          max_prob_list.append(j)
      samples = np.random.multinomial(20,[max_prob]*len(max_prob_list),size=1)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(max_prob_list[index])
  else:
    sentence.extend(initial_sequence)
    i = len(initial_sequence)
    temp=0
    while(sentence[-1]!="</s>" and i<20):
      max_prob = 0
      max_prob_list = list()
      for j in vocab_dict:
        word_list = sentence[-n_gram+1:]
        word_list.append(j)
        if(n_gram==2):
          k = big(word_list)
        elif(n_gram==3):
          k = trig(word_list)
        elif(n_gram==4):
          k = quadg(word_list)
        if(k>max_prob):
          max_prob = k
          max_prob_list = [j]
        elif(k==max_prob):
          max_prob_list.append(j)
      samples = np.random.multinomial(20,[max_prob]*len(max_prob_list),size=1)
      max_len=len(max_prob_list)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(max_prob_list[rd(0,max_len,1)])
      i+=1
  
  return sentence

In [157]:
# GENERATING RANDOM TEXT:
# UNIGRAMS
print("UNIGRAM: ", end = " ")
print(" ".join(MLE(1,[])))
# BIGRAM
print("BIGRAM: ", end = " ")
print(" ".join(MLE(2,["it", "is"])))
# TRIGRAM
print("TRIGRAM: ", end = " ")
print(" ".join(MLE(3,["there", "is", "something"])))
# QUADGRAM
print("QUADGRAM1: ", end = " ")
print(" ".join(MLE(4,["but", "there", "is", "no"])))
# ANOTHER QuADGRAM


UNIGRAM:  <s> the the the the the the the the the the the the the the the the the the the the
BIGRAM:  <s> it is a lot of the way I think of the way I think of the way I think of
TRIGRAM:  <s> there is something plans someone viciously decisions fricking still Robin Sir wars debates Defense lawyer ACHIEVEMENT LAW HIGH auctioned hitting
QUADGRAM1:  <s> but there is no thrilled hand prefer damage Highly complicated CALLED richest REPRESENT Todd Half stated Street comfortable INDIRECTLY impression


The generated text is human readable, but isn't grammatically perfect

# Neural Approach 

Using TensorFlow backend.
